# import libs

In [ ]:
import os
import socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from FCNN import TwoInputFCNN as FCNN


# import data

In [ ]:
# paths pre-setting
if 'gput' in socket.gethostname() or 'hpc' in socket.gethostname():
    label_root = '/home/sxy841/ERIE/silicone/output'
    model_root = '/home/sxy841/ERIE/silicone/Track-Shuyuan-2023-06-13/videos'
else:
    label_root = '/home/ysy/ERIE/single_psm_manipulation_dataset/output'
    model_root = '/home/ysy/ERIE/videos'

model_name = 'DLC_resnet50_TrackJun13shuffle1_50000'



In [ ]:
# pre setting training sets
training_sets = ['C_M1_T1_1', 'C_M1_T1_3', 'C_M1_T1_5', 'C_M1_T1_7', 
                 'R2_M1_T1_1', 'R2_M1_T1_3', 'R2_M1_T1_5', 'R2_M1_T1_7', 
                 'L2_M1_T1_1', 'L2_M1_T1_3', 'L2_M1_T1_5', 'L2_M1_T1_7', 
                 'Z2_M1_T1_1', 'Z2_M1_T1_3', 'Z2_M1_T1_5']

# initial arrays
X_train_L = np.zeros((0, 16))
X_train_R = np.zeros((0, 16))
y_train = np.zeros((0, 3))

for set in tqdm(training_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    X_train_L = np.vstack((X_train_L,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    X_train_R = np.vstack((X_train_R,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    y_train = np.vstack((y_train, labels[:frames, 6:9]))


from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
X_scaler = MinMaxScaler()
X_train_L = X_scaler.fit_transform(X_train_L)
X_train_R = X_scaler.transform(X_train_R)

y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)



In [ ]:
X_train_L.shape, X_train_R.shape, y_train.shape

In [ ]:
# pre setting validation sets
test_sets = ['C_M1_T1_2', 'C_M1_T1_4',
             'R2_M1_T1_2', 'R2_M1_T1_4',
             'L2_M1_T1_2', 'L2_M1_T1_4',
             'Z2_M1_T1_2',]

# initial arrays
X_val_L = np.zeros((0, 16))
X_val_R = np.zeros((0, 16))
y_val = np.zeros((0, 3))

for set in tqdm(test_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    X_val_L = np.vstack((X_val_L,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    X_val_R = np.vstack((X_val_R,
                          coordinates_L.filter(
                              regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]))
    y_val = np.vstack((y_val, labels[:frames, 6:9]))

X_val_L = X_scaler.transform(X_val_L)
X_val_R = X_scaler.transform(X_val_R)
y_val = y_scaler.transform(y_val)


In [ ]:
X_val_L.shape, X_val_R.shape, y_val.shape

# Trainning and validation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for i in range(2,7,2):
    print(i)
    net = FCNN(input_dim=X_train_L.shape[1], hidden_dim=64, output_dim=y_train.shape[1], device=device, num_hidden_layers=i, batch_size=32)
    net.train(X_train_R, X_train_L, y_train, X_val_R, X_val_L, y_val, epochs=100, use_tqdm=False, save_loss=True)


# Demos

In [ ]:
import json
import matplotlib.pyplot as plt
import glob
import numpy as np

def plot_loss(filter_params=None):
    # Get a list of files that match the pattern
    files = glob.glob('results/params_and_losses*.json')

    # Dimension names
    dim_names = ['x', 'y', 'z', 'Mean']

    # Determine the number of subplots from the loss dimension
    with open(files[0], 'r') as f:
        data = json.load(f)
    losses = data['losses'][0]  # Take first loss array as an example
    num_subplots = len(losses)  # Include plot for the mean loss

    # Create a subplot for each loss dimension
    fig, axs = plt.subplots(num_subplots, figsize=(12, 6 * num_subplots))

    # Ensure axs is a list even when num_subplots = 1
    if num_subplots == 1:
        axs = [axs]

    # List to store parameters of the files that passed the filter
    filtered_params_list = []

    # Loop over each file
    for filename in files:
        # Load the data from the JSON file
        with open(filename, 'r') as f:
            data = json.load(f)

        # Extract the training parameters from the data
        params = {k: v for k, v in data.items() if k != 'losses'}

        # If filter is not None, skip this file if it does not match the filter
        if filter_params is not None and not all(item in params.items() for item in filter_params.items()):
            continue

        # If the file passed the filter, add its parameters to the list
        filtered_params_list.append(params)

    # Determine common parameters
    common_params = filtered_params_list[0].copy()
    for params in filtered_params_list[1:]:
        keys_to_delete = []
        for k in common_params.keys():
            if common_params[k] != params[k]:
                keys_to_delete.append(k)
        for k in keys_to_delete:
            common_params.pop(k)

    # Loop over each file again to plot the data
    for i, filename in enumerate(files):
        # Load the data from the JSON file
        with open(filename, 'r') as f:
            data = json.load(f)

        # Extract the unique training parameters for this file
        unique_params = {k: v for k, v in filtered_params_list[i].items() if k not in common_params}

        # Create a label for this line from the unique training parameters
        label = ', '.join(f'{key}: {value}' for key, value in unique_params.items())

        # Extract loss values
        losses = data['losses']

        # Plot the loss values for each dimension
        for j in range(num_subplots):
            axs[j].plot([loss[j] for loss in losses], label=label)
            axs[j].set_title(f'Loss for {dim_names[j]} over iterations')
            axs[j].set_xlabel('Iteration')
            axs[j].set_ylabel('Loss')
            axs[j].grid(True)
            axs[j].legend()

    plt.tight_layout()
    plt.show()


In [ ]:
plot_loss(filter_params={"lr": 0.001, "batch_size": 32})

In [ ]:
plot_loss(filter_params={"num_hidden_layers": 2})